In [1]:
import numpy as np
import numpy as np
import pandas as pd
import WindFarmGenetic # wind farm layout optimization using genetic algorithms classes
from datetime import datetime
import os
from sklearn.svm import SVR
import pickle


In [2]:
import os
os.getcwd()

'/home/naveen/arena/hackathon/shell/hackathon/python/Ver0_N'

In [3]:
##
wt_N=50  #number of turbines
NA_loc=[]  #index of obstructed locations
population_size=100  #population size/number of layouts for genetic algorithms
iteration_times=100  #number of iterations

# wind farm size, cells
cols_cells = 10  # number of cells each row
rows_cells = 10  # number of cells each column
cell_width = 400 # unit (TBD)--> have to be computed.

#genetic algorithmic parameters.
elite_rate = 0.2 
cross_rate = 0.6
mutate_rate = 0.1

random_rate = 0.5
num_runs=100


In [4]:
data_folder = "naveen_data" #temporary code related files. (MOdify it as per your specification)
svr_model_filename = 'svr_1s1d_N_13.svr' #svr training file.
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

wds_data_folder = "{}/wds".format(data_folder)
if not os.path.exists(wds_data_folder):
    os.makedirs(wds_data_folder)
    
# initial populations saved folder
init_pops_data_folder = "{}/init_data".format(data_folder)
if not os.path.exists(init_pops_data_folder):
    os.makedirs(init_pops_data_folder)

#result folder
results_data_folder = data_folder+"/results"
if not os.path.exists(results_data_folder):
    os.makedirs(results_data_folder)
    
#result folder for sugga genetic algorithm
sg_result_folder = "{}/sg".format(results_data_folder)
if not os.path.exists(sg_result_folder):
    os.makedirs(sg_result_folder)

In [5]:
import WindFarmGenetic
wfg = WindFarmGenetic.WindFarmGenetic(rows=rows_cells, cols=cols_cells, N=wt_N, NA_loc=NA_loc, pop_size=population_size,
                                      iteration=iteration_times,cell_width=cell_width, elite_rate=elite_rate,
                                             cross_rate=cross_rate, random_rate=random_rate, mutate_rate=mutate_rate)

In [6]:
for i in range(num_runs):
    wfg.gen_init_pop_NA()
    wfg.save_init_pop_NA("{}/init_{}.dat".format(init_pops_data_folder, i),
                         "{}/init_{}_NA.dat".format(init_pops_data_folder, i))

In [7]:
##Generate initial layouts: single layout--> 1d array of size row*col, layouts are set of population_size different(random) layouts.  
train_mc_layouts, train_mc_layouts_NA = WindFarmGenetic.LayoutGridMCGenerator.gen_mc_grid_with_NA_loc(rows_cells,
                                                                                                      cols_cells,
                                                                                                      population_size,
                                                                                                      wt_N, NA_loc,
                                                                                                      "{}/mc_layout.dat".format(wds_data_folder),
                                                                                                      "{}/mc_layout_NA.dat".format(wds_data_folder))

layouts = np.genfromtxt("{}/mc_layout.dat".format(wds_data_folder), delimiter="  ", dtype=np.int32)

In [8]:
import WindFarmGenetic

wfg.mc_gen_xy_NA_hackathon(rows=rows_cells, cols=cols_cells, layouts=layouts, n=population_size, N=wt_N,  xfname="{}/x.dat".format(wds_data_folder),
                 yfname="{}/y.dat".format(wds_data_folder))

In [9]:
x_original = pd.read_csv("{}/x.dat".format(wds_data_folder), header=None, nrows=rows_cells * cols_cells,
                         delim_whitespace=True, dtype=np.float32)
x_original = x_original.values

In [10]:
y_original = pd.read_csv("{}/y.dat".format(wds_data_folder), header=None, nrows=rows_cells * cols_cells,
                         delim_whitespace=True, dtype=np.float32)
y_original = y_original.values.flatten()

In [11]:
# create a SVR object and specify the kernal and other parameters
svr_model = SVR(kernel='rbf', C=2000.0, gamma=0.3, epsilon=.1)
# build the SVR power distribution model
svr_model.fit(x_original, y_original)

# save the SVR model to a file
pickle.dump(svr_model, open("{}/{}".format(wds_data_folder, svr_model_filename), 'wb'))

In [12]:
# SUGGA: support vector regression guided genetic algorithm

for i in range(0, num_runs):  # run times
    print("run number: {} ...".format(i))
    wfg.load_init_pop_NA("{}/init_{}.dat".format(init_pops_data_folder, i),
                         "{}/init_{}_NA.dat".format(init_pops_data_folder, i))
    run_time, eta = wfg.sugga_genetic_alg(i, svr_model=svr_model, result_folder=sg_result_folder)
    result_arr[i, 0] = run_time
    result_arr[i, 1] = eta
time_stamp = datetime.now().strftime("%Y%m%d%H%M%S")
filename = "{}/result_sugga_{}.dat".format(sg_result_folder, time_stamp)
np.savetxt(filename, result_arr, fmt='%f', delimiter="  ")

run number: 0 ...
Support vector regression guided genetic algorithm starts....
generation 0...
[541.67456 533.7436  534.85675 535.68427 534.73346 539.59106 540.4199
 540.1531  538.2676  532.54034 542.29553 538.11426 540.7936  541.1494
 540.85913 541.78735 543.9427  541.47754 537.4409  535.34937 539.93945
 539.8547  536.33746 537.0982  535.7021  539.5483  542.82086 535.5099
 540.94586 537.9409  541.9299  539.3912  540.039   541.5063  541.59174
 534.29675 542.9216  530.4157  536.7034  539.32074 536.37714 536.1521
 540.45544 539.9311  540.3817  539.86127 537.0115  538.0035  537.7589
 538.08026 536.4162  534.98926 540.02606 541.2915  532.31915 536.9288
 540.66406 539.4561  541.54706 540.7579  536.19055 538.5942  540.739
 541.0846  536.9914  536.99884 539.8302  538.81067 538.72296 531.25323
 537.21765 537.7861  536.3819  538.13916 537.68475 538.6163  535.96094
 541.806   536.0299  540.2787  535.1943  537.877   538.5326  535.70465
 538.4747  536.0138  542.5063  539.7652  542.0405  542.01074

KeyboardInterrupt: 

### Shell Power Computation Package

In [1]:
import sys 
sys.path.append('../Shell_Code_Modified/')

In [2]:
import Farm_Evaluator_Vec

In [3]:
total_energy, turbine_power=Farm_Evaluator_Vec.get_AEP_results()

Both perimeter and proximity constraints are satisfied !!

power
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.18937522 0.18937522 0.18937522 ... 0.09336138 0.18937522 0.06266943]
 ...
 [3.         3.         3.         ... 3.         3.         3.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
(540, 50)


In [4]:
total_energy

545.5921376037597